In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

df = pd.read_csv(r"C:\Users\Lenovo\Desktop\24-25 bahar\ADL\ADL_project\df_last_encoded.csv")  # sen zaten yüklüyorsan atla

# 'designation' veya ilgili kolonun adını bul
designation_col = None
for cname in df.columns:
    if 'designation' in cname.lower() or 'mat' in cname.lower():
        designation_col = cname
        break

if designation_col is None:
    raise ValueError("Designation/mat0/mat1 kolonu bulunamadı!")

# Eğer string ise label encode et
if df[designation_col].dtype == 'object' or str(df[designation_col].dtype).startswith('str'):
    le = LabelEncoder()
    df[designation_col + '_label'] = le.fit_transform(df[designation_col])
    designation_feat = designation_col + '_label'
else:
    designation_feat = designation_col

# Hedefler
feature_cols = [col for col in df.columns if col not in ['n_opt', 'pce', 'hopt (%)', 'PCE (%)']]
if designation_feat not in feature_cols:
    feature_cols.append(designation_feat)  # Designation'ı featurelara ekle
X = df[feature_cols]
y_nopt = df['hopt (%)']

# Son 30 satırı test olarak ayır
X_train, X_test = X.iloc[:-30, :], X.iloc[-30:, :]
y_train, y_test = y_nopt.iloc[:-30], y_nopt.iloc[-30:]

# Sütun isimlerini sırayla X0, X1,... olarak göster
for i, col in enumerate(feature_cols):
    print(f"X{i}: {col}")


X0: Designation
X1: Uzunluk
X2: Genislik
X3: Kalinlik
X4: mat0
X5: mat1
X6: abs_peak
X7: abs_min
X8: abs_max
X9: em_peak
X10: em_min
X11: em_max
X12: QY (%)


In [ ]:
import optuna
from gplearn.genetic import SymbolicRegressor
from sklearn.metrics import mean_squared_error

def objective(trial):
    p_crossover        = trial.suggest_float('p_crossover', 0.5, 0.75)
    p_subtree_mutation = trial.suggest_float('p_subtree_mutation', 0.05, 0.12)
    p_hoist_mutation   = trial.suggest_float('p_hoist_mutation', 0.01, 0.07)
    p_point_mutation   = trial.suggest_float('p_point_mutation', 0.05, 0.12)
    if (p_crossover + p_subtree_mutation + p_hoist_mutation + p_point_mutation) > 1.0:
        raise optuna.exceptions.TrialPruned()

    sr = SymbolicRegressor(
        population_size=trial.suggest_int('population_size', 800, 10000, step=200),
        generations=trial.suggest_int('generations', 10, 100, step=5),
        stopping_criteria=0.001,
        p_crossover=p_crossover,
        p_subtree_mutation=p_subtree_mutation,
        p_hoist_mutation=p_hoist_mutation,
        p_point_mutation=p_point_mutation,
        max_samples=trial.suggest_float('max_samples', 0.8, 1.0),
        parsimony_coefficient=trial.suggest_float('parsimony_coefficient', 0.001, 0.03, log=True),
        function_set=['add', 'sub', 'mul', 'div', 'sin', 'cos','sqrt','log'],  # hızlı prototip için sade set!
        metric='mse',
        init_depth=trial.suggest_categorical('init_depth', [(2,5), (3,6), (2,6)]),
        init_method=trial.suggest_categorical('init_method', ['half and half', 'grow']),
        const_range=None,
        verbose=0,
        random_state=42
    )
    sr.fit(X_train, y_train)
    y_pred = sr.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    return mse  # minimize

# HIZLI OPTIMIZASYON (düşük trial)
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=80, show_progress_bar=True)  # n_trials: 5-10 arası hızlı sonuç

print("\nBest hyperparameters:")
for key, value in study.best_params.items():
    print(f"{key}: {value}")

# EN İYİ PARAMETRELERLE MODELİ TEKRAR EĞİT
best_params = study.best_params

sr = SymbolicRegressor(
    population_size=best_params['population_size'],
    generations=best_params['generations'],
    stopping_criteria=0.001,
    p_crossover=best_params['p_crossover'],
    p_subtree_mutation=best_params['p_subtree_mutation'],
    p_hoist_mutation=best_params['p_hoist_mutation'],
    p_point_mutation=best_params['p_point_mutation'],
    max_samples=best_params['max_samples'],
    parsimony_coefficient=best_params['parsimony_coefficient'],
    function_set=['add', 'sub', 'mul', 'div','sin','cos','log','sqrt'],  # yukarıdakiyle aynı set
    metric='mse',
    init_depth=best_params['init_depth'],
    init_method=best_params['init_method'],
    const_range=None,
    verbose=1,
    random_state=42
)

sr.fit(X_train, y_train)

# Sonra sr ile tüm tahmin ve analiz işlemlerini yapabilirsin:
# y_pred_train = sr.predict(X_train)
# y_pred_test  = sr.predict(X_test)



c:\Users\Lenovo\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2025-10-02 22:20:05,581] A new study created in memory with name: no-name-f5b9b7cc-a419-43ee-ac96-8fa767d11e4b
  0%|          | 0/80 [00:00<?, ?it/s]c:\Users\Lenovo\AppData\Local\Programs\Python\Python39\lib\site-packages\optuna\distributions.py:515: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains (2, 5) which is of type tuple.
  warnings.warn(message)
c:\Users\Lenovo\AppData\Local\Programs\Python\Python39\lib\site-packages\optuna\distributions.py:515: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains (3, 6) which is of type tuple.

[I 2025-10-02 22:22:54,598] Trial 0 finished with value: 0.2778532230627197 and parameters: {'p_crossover': 0.6828639087359278, 'p_subtree_mutation': 0.05599662353685968, 'p_hoist_mutation': 0.0637271218382036, 'p_point_mutation': 0.09853632470109983, 'population_size': 9600, 'generations': 30, 'max_samples': 0.9024886813158493, 'parsimony_coefficient': 0.029151269937195416, 'init_depth': (3, 6), 'init_method': 'grow'}. Best is trial 0 with value: 0.2778532230627197.


Best trial: 0. Best value: 0.277853:   2%|▎         | 2/80 [03:07<1:44:35, 80.45s/it] c:\Users\Lenovo\AppData\Local\Programs\Python\Python39\lib\site-packages\optuna\distributions.py:515: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains (2, 5) which is of type tuple.
  warnings.warn(message)
c:\Users\Lenovo\AppData\Local\Programs\Python\Python39\lib\site-packages\optuna\distributions.py:515: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains (3, 6) which is of type tuple.
  warnings.warn(message)
c:\Users\Lenovo\AppData\Local\Programs\Python\Python39\lib\site-packages\optuna\distributions.py:515: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains (2, 6) which is of type tuple.
  warnings.warn(message)
c:\Users\Lenovo\AppD

[I 2025-10-02 22:23:13,054] Trial 1 finished with value: 0.466591812894055 and parameters: {'p_crossover': 0.6353487691122441, 'p_subtree_mutation': 0.07795953339111614, 'p_hoist_mutation': 0.06346808313687713, 'p_point_mutation': 0.05546663218333738, 'population_size': 2600, 'generations': 20, 'max_samples': 0.9701585345819594, 'parsimony_coefficient': 0.003403390892102365, 'init_depth': (2, 5), 'init_method': 'grow'}. Best is trial 0 with value: 0.2778532230627197.


Best trial: 0. Best value: 0.277853:   4%|▍         | 3/80 [03:20<1:03:55, 49.82s/it]c:\Users\Lenovo\AppData\Local\Programs\Python\Python39\lib\site-packages\optuna\distributions.py:515: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains (2, 5) which is of type tuple.
  warnings.warn(message)
c:\Users\Lenovo\AppData\Local\Programs\Python\Python39\lib\site-packages\optuna\distributions.py:515: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains (3, 6) which is of type tuple.
  warnings.warn(message)
c:\Users\Lenovo\AppData\Local\Programs\Python\Python39\lib\site-packages\optuna\distributions.py:515: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains (2, 6) which is of type tuple.
  warnings.warn(message)
c:\Users\Lenovo\AppDa

...

[I 2025-10-03 07:28:57,025] Trial 66 finished with value: 0.1567839359611224 and parameters: {'p_crossover': 0.7175615038788066, 'p_subtree_mutation': 0.08242297366455748, 'p_hoist_mutation': 0.03458786726901465, 'p_point_mutation': 0.0698114498721363, 'population_size': 8600, 'generations': 100, 'max_samples': 0.86124799258781, 'parsimony_coefficient': 0.0014330060583524556, 'init_depth': (2, 6), 'init_method': 'half and half'}. Best is trial 60 with value: 0.07542198209962676.


Best trial: 60. Best value: 0.075422:  85%|████████▌ | 68/80 [9:15:34<2:38:48, 794.02s/it]c:\Users\Lenovo\AppData\Local\Programs\Python\Python39\lib\site-packages\optuna\distributions.py:515: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains (2, 5) which is of type tuple.
  warnings.warn(message)
c:\Users\Lenovo\AppData\Local\Programs\Python\Python39\lib\site-packages\optuna\distributions.py:515: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains (3, 6) which is of type tuple.
  warnings.warn(message)
c:\Users\Lenovo\AppData\Local\Programs\Python\Python39\lib\site-packages\optuna\distributions.py:515: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains (2, 6) which is of type tuple.
  warnings.warn(message)
c:\Users\Lenovo\

[I 2025-10-03 07:35:39,948] Trial 67 finished with value: 0.5586367067918782 and parameters: {'p_crossover': 0.6669741256786578, 'p_subtree_mutation': 0.05835870382729075, 'p_hoist_mutation': 0.043405937749864114, 'p_point_mutation': 0.05889275312623964, 'population_size': 9000, 'generations': 45, 'max_samples': 0.8507189628231071, 'parsimony_coefficient': 0.0026685166028072855, 'init_depth': (3, 6), 'init_method': 'half and half'}. Best is trial 60 with value: 0.07542198209962676.


Best trial: 60. Best value: 0.075422:  86%|████████▋ | 69/80 [9:27:15<2:20:26, 766.09s/it]c:\Users\Lenovo\AppData\Local\Programs\Python\Python39\lib\site-packages\optuna\distributions.py:515: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains (2, 5) which is of type tuple.
  warnings.warn(message)
c:\Users\Lenovo\AppData\Local\Programs\Python\Python39\lib\site-packages\optuna\distributions.py:515: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains (3, 6) which is of type tuple.
  warnings.warn(message)
c:\Users\Lenovo\AppData\Local\Programs\Python\Python39\lib\site-packages\optuna\distributions.py:515: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains (2, 6) which is of type tuple.
  warnings.warn(message)
c:\Users\Lenovo\

[I 2025-10-03 07:47:20,847] Trial 68 finished with value: 0.20120572696036954 and parameters: {'p_crossover': 0.6073263161224477, 'p_subtree_mutation': 0.07722848074579187, 'p_hoist_mutation': 0.04780907873923061, 'p_point_mutation': 0.07377353203037139, 'population_size': 6600, 'generations': 95, 'max_samples': 0.8744508512161048, 'parsimony_coefficient': 0.001861593311894082, 'init_depth': (3, 6), 'init_method': 'half and half'}. Best is trial 60 with value: 0.07542198209962676.


In [ ]:
# optunasız GA
"""from gplearn.genetic import SymbolicRegressor

sr = SymbolicRegressor(
    population_size=10000,
    generations=100,
    stopping_criteria=0.001,
    p_crossover=0.7,
    p_subtree_mutation=0.1,
    p_hoist_mutation=0.05,
    p_point_mutation=0.1,
    max_samples=0.95,
    parsimony_coefficient=0.003,
    function_set=['add', 'sub', 'mul', 'div', 'sqrt', 'log', 'sin', 'cos'],
    metric='mse',
    init_depth=(3, 8),
    init_method='half and half',
    const_range=None,
    verbose=1,
    random_state=42
)
sr.fit(X_train, y_train)

"""


In [ ]:
from gplearn.genetic import SymbolicRegressor
from sklearn.metrics import mean_squared_error

# ... (optuna ile hyperparam tuning ve training kodun aynen)

# Eğitim sonrası
print("Bulunan formül:", sr._program)

# Formülde Xn (designation’ın indexi) var mı?
designation_idx = feature_cols.index(designation_feat)
if f"X{designation_idx}" in str(sr._program):
    print(f"Formülde designation (X{designation_idx}) KULLANILDI!")
else:
    print("Formülde designation YOK! (Model bunu matematiksel olarak gerek görmedi.)")


In [ ]:
# Bulunan matematiksel formül
print("n_opt için bulunan formül:")
print(sr._program)


In [ ]:
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import numpy as np

y_pred_train = sr.predict(X_train)
y_pred_test = sr.predict(X_test)

# Metrikler
print("Train R2:", r2_score(y_train, y_pred_train))
print("Test R2 :", r2_score(y_test, y_pred_test))
print("Test MSE:", mean_squared_error(y_test, y_pred_test))
print("Test MAE:", mean_absolute_error(y_test, y_pred_test))


In [ ]:
from sklearn.model_selection import cross_val_score

cv_scores = cross_val_score(sr, X_train, y_train, cv=5, scoring='r2')
print("5-Fold CV R2 Skorları (Train Seti):", cv_scores)
print("CV Ortalama R2:", np.mean(cv_scores))


In [ ]:
"""
Symbolic Regression - Comprehensive Analysis
Optik Malzeme n_opt Tahmini - Görselleştirme ve Analiz
"""

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import pearsonr
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from collections import Counter

# Seaborn style ayarları
sns.set_theme(style="whitegrid", font_scale=1.25, rc={"axes.labelweight":"bold"})

In [ ]:
# ====================================
# DATA PREPARATION & UTILITY FUNCTIONS
# ====================================

def to_numpy(X):
    """Convert to numpy array safely"""
    return X.values if hasattr(X, "values") else np.asarray(X)

def get_feature_names(X):
    """Extract feature names"""
    if hasattr(X, "columns"):
        return list(X.columns)
    X_np = to_numpy(X)
    return [f"X{i}" for i in range(X_np.shape[1])]

def calculate_metrics(y_true, y_pred):
    """Calculate all regression metrics"""
    corr, p_value = pearsonr(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    return {
        'correlation': corr,
        'p_value': p_value,
        'r2': r2,
        'mse': mse,
        'mae': mae
    }

# Veri hazırlığı
y_train_plot = pd.Series(y_train).reset_index(drop=True)
y_pred_train_plot = pd.Series(y_pred_train).reset_index(drop=True)
y_test_plot = pd.Series(y_test).reset_index(drop=True)
y_pred_test_plot = pd.Series(y_pred_test).reset_index(drop=True)

Xtr = to_numpy(X_train)
Xte = to_numpy(X_test)
feat_names = get_feature_names(X_train)

In [ ]:
# ====================================
# VISUALIZATION FUNCTIONS
# ====================================

def plot_scatter_comparison(y_train, y_pred_train, y_test, y_pred_test):
    """Combined train-test scatter plot"""
    fig, ax = plt.subplots(1, 2, figsize=(12, 6))
    
    # Train scatter
    sns.scatterplot(x=y_train, y=y_pred_train, s=65, color="#3288bd", 
                   edgecolor='k', ax=ax[0])
    ax[0].plot([y_train.min(), y_train.max()], [y_train.min(), y_train.max()], 
               'r--', lw=2, label="Identity (y = x)")
    ax[0].set_xlabel("Actual $n_{opt}$ (Train Set)")
    ax[0].set_ylabel("Predicted $n_{opt}$ (Train Set)")
    ax[0].set_title("Train Set: Actual vs. Predicted")
    ax[0].legend()
    ax[0].grid(True, linestyle=':', alpha=0.7)
    
    # Test scatter
    sns.scatterplot(x=y_test, y=y_pred_test, s=65, color="#e08214", 
                   edgecolor='k', ax=ax[1])
    ax[1].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 
               'r--', lw=2, label="Identity (y = x)")
    ax[1].set_xlabel("Actual $n_{opt}$ (Test Set)")
    ax[1].set_ylabel("Predicted $n_{opt}$ (Test Set)")
    ax[1].set_title("Test Set: Actual vs. Predicted")
    ax[1].legend()
    ax[1].grid(True, linestyle=':', alpha=0.7)
    
    plt.tight_layout()
    plt.show()

def plot_residual_analysis(y_test, y_pred_test):
    """Residual plot for test set"""
    residuals = y_test - y_pred_test
    plt.figure(figsize=(8, 4.5))
    sns.scatterplot(x=y_pred_test, y=residuals, s=95, color="#e08214", edgecolor='k')
    plt.axhline(0, color='red', linestyle='--', lw=2, label="Zero Error")
    plt.xlabel("Predicted $n_{opt}$ (Test Set)")
    plt.ylabel("Residual (Actual - Predicted)")
    plt.title("Test Set: Residual Analysis")
    plt.legend()
    plt.tight_layout()
    plt.show()

def plot_time_series_comparison(y_test, y_pred_test):
    """Time series comparison plot"""
    plt.figure(figsize=(13, 5))
    sns.lineplot(x=np.arange(len(y_test)), y=y_test, label='Actual $n_{opt}$', 
                lw=2.5, color='#1f77b4')
    sns.lineplot(x=np.arange(len(y_pred_test)), y=y_pred_test, 
                label='Predicted $n_{opt}$', lw=2.5, color='#ff7f0e')
    plt.fill_between(np.arange(len(y_test)), y_test, y_pred_test, 
                     color='grey', alpha=0.18, label='Error Area')
    plt.xlabel("Sample Index (Test Set)")
    plt.ylabel("$n_{opt}$ Value")
    plt.title("Test Set: Actual vs Predicted Comparison")
    plt.legend()
    plt.tight_layout()
    plt.show()

def plot_error_histogram(y_test, y_pred_test):
    """Absolute error histogram"""
    abs_error = np.abs(y_test - y_pred_test)
    plt.figure(figsize=(7, 4))
    sns.histplot(abs_error, bins=8, kde=True, color='#5dade2', 
                edgecolor='k', alpha=0.9)
    plt.xlabel("Absolute Error")
    plt.ylabel("Frequency")
    plt.title("Test Set: Absolute Error Distribution")
    plt.tight_layout()
    plt.show()

In [ ]:
# ====================================
# METRICS CALCULATION & REPORTING
# ====================================

def print_metrics(y_train, y_pred_train, y_test, y_pred_test):
    """Calculate and print all metrics"""
    
    train_metrics = calculate_metrics(y_train, y_pred_train)
    test_metrics = calculate_metrics(y_test, y_pred_test)
    
    print("="*50)
    print("SYMBOLIC REGRESSION PERFORMANCE METRICS")
    print("="*50)
    
    print("\nTRAIN SET METRICS:")
    print(f"Pearson correlation: {train_metrics['correlation']:.4f} (p-value: {train_metrics['p_value']:.2e})")
    print(f"R² score           : {train_metrics['r2']:.4f}")
    print(f"MSE                : {train_metrics['mse']:.4f}")
    print(f"MAE                : {train_metrics['mae']:.4f}")
    
    print("\nTEST SET METRICS:")
    print(f"Pearson correlation: {test_metrics['correlation']:.4f} (p-value: {test_metrics['p_value']:.2e})")
    print(f"R² score           : {test_metrics['r2']:.4f}")
    print(f"MSE                : {test_metrics['mse']:.4f}")
    print(f"MAE                : {test_metrics['mae']:.4f}")
    print("="*50)
    
    return train_metrics, test_metrics

In [ ]:
# ====================================
# ADVANCED ANALYSIS FUNCTIONS
# ====================================

def numerical_partials(estimator, X, eps=1e-4):
    """Numerical partial derivatives using central difference"""
    X = np.asarray(X, dtype=float)
    n, d = X.shape
    grads = np.zeros((n, d), dtype=float)
    
    for j in range(d):
        Xp = X.copy()
        Xm = X.copy()
        h = eps * (np.abs(X[:, j]) + 1.0)
        Xp[:, j] += h
        Xm[:, j] -= h
        yp = estimator.predict(Xp)
        ym = estimator.predict(Xm)
        grads[:, j] = (yp - ym) / (2.0 * h)
    
    return grads

def plot_sensitivity_analysis(estimator, X_train, feat_names, top_k=10):
    """Feature sensitivity analysis using partial derivatives"""
    
    grads_train = numerical_partials(estimator, X_train, eps=1e-4)
    mean_abs_grad = np.mean(np.abs(grads_train), axis=0)
    order = np.argsort(-mean_abs_grad)
    topk = min(top_k, len(feat_names))
    top_idx = order[:topk]
    
    # Bar plot
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.bar([feat_names[i] for i in top_idx], mean_abs_grad[top_idx])
    plt.ylabel("Mean |∂ŷ/∂x|")
    plt.title(f"Feature Sensitivity (Top {topk})")
    plt.xticks(rotation=45, ha="right")
    
    # Heatmap
    plt.subplot(1, 2, 2)
    plt.imshow(grads_train[:, top_idx].T, aspect='auto', interpolation='nearest')
    plt.colorbar(label="∂ŷ/∂x")
    plt.yticks(range(topk), [feat_names[i] for i in top_idx])
    plt.xlabel("Sample Index")
    plt.title("Sensitivity Heatmap")
    
    plt.tight_layout()
    plt.show()
    
    return top_idx, mean_abs_grad

def pdp_1d(estimator, X, j, grid_resolution=40, q_low=0.01, q_high=0.99):
    """Partial Dependence Plot for single feature"""
    X = np.asarray(X, dtype=float)
    xj = X[:, j]
    lo = np.quantile(xj, q_low)
    hi = np.quantile(xj, q_high)
    grid = np.linspace(lo, hi, grid_resolution)
    
    pdp_vals = []
    for v in grid:
        Xtmp = X.copy()
        Xtmp[:, j] = v
        yhat = estimator.predict(Xtmp)
        pdp_vals.append(yhat.mean())
    
    return grid, np.array(pdp_vals)

def plot_pdp_analysis(estimator, X_train, feat_names, top_indices, n_features=3):
    """Plot PDP for top features"""
    n_plot = min(n_features, len(top_indices))
    
    fig, axes = plt.subplots(1, n_plot, figsize=(5*n_plot, 4))
    if n_plot == 1:
        axes = [axes]
    
    for i, j in enumerate(top_indices[:n_plot]):
        grid, pdp_vals = pdp_1d(estimator, X_train, j)
        axes[i].plot(grid, pdp_vals, lw=2, color='#2E86AB')
        axes[i].set_xlabel(feat_names[j])
        axes[i].set_ylabel("E[ŷ | x_j]")
        axes[i].set_title(f"PDP: {feat_names[j]}")
        axes[i].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

In [ ]:
# ====================================
# BOOTSTRAP STABILITY ANALYSIS
# ====================================

def clone_sr_with_params(base_sr, random_state):
    """Clone SR estimator with new random state"""
    params = base_sr.get_params(deep=True)
    params['random_state'] = random_state
    params['verbose'] = 0
    return type(base_sr)(**params)

def bootstrap_stability_analysis(estimator, X_train, y_train, X_test, y_test, 
                                n_bootstrap=30, random_seed=123):
    """Bootstrap analysis for model stability"""
    
    expr_list = []
    complexities = []
    Yhat_te = []
    
    rng = np.random.default_rng(random_seed)
    
    print(f"Running {n_bootstrap} bootstrap iterations...")
    
    for b in range(n_bootstrap):
        idx = rng.integers(0, X_train.shape[0], size=X_train.shape[0])
        Xb = X_train[idx]
        yb = to_numpy(y_train)[idx]
        
        sr_b = clone_sr_with_params(estimator, random_state=42 + b)
        sr_b.fit(Xb, yb)
        
        prog = getattr(sr_b, "_program", None)
        expr = str(prog) if prog is not None else None
        expr_list.append(expr)
        
        length = getattr(prog, "length_", None)
        depth = getattr(prog, "depth_", None)
        complexities.append({"length": length, "depth": depth})
        
        yhat = sr_b.predict(X_test)
        Yhat_te.append(yhat)
        
        if (b + 1) % 10 == 0:
            print(f"  Completed {b + 1}/{n_bootstrap}")
    
    Yhat_te = np.vstack(Yhat_te)
    return expr_list, complexities, Yhat_te

def plot_bootstrap_results(y_test, Yhat_te, expr_list, complexities):
    """Plot bootstrap analysis results"""
    
    # Prediction bands
    low = np.percentile(Yhat_te, 5, axis=0)
    med = np.percentile(Yhat_te, 50, axis=0)
    high = np.percentile(Yhat_te, 95, axis=0)
    
    plt.figure(figsize=(12, 5))
    plt.plot(range(len(y_test)), to_numpy(y_test), label="Actual", lw=2, color='#1f77b4')
    plt.plot(range(len(med)), med, label="Bootstrap Median", lw=2, color='#ff7f0e')
    plt.fill_between(range(len(low)), low, high, alpha=0.25, label="5%-95% Band", color='#ff7f0e')
    plt.xlabel("Sample Index (Test)")
    plt.ylabel("$n_{opt}$")
    plt.title("Bootstrap Prediction Uncertainty")
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    # Expression frequency
    ctr = Counter(expr_list)
    print("\nMost frequent expressions:")
    for expr, cnt in ctr.most_common(5):
        print(f"{cnt:>3}x  {expr}")
    
    # Complexity distribution
    lengths = [c["length"] for c in complexities if c["length"] is not None]
    depths = [c["depth"] for c in complexities if c["depth"] is not None]
    
    plt.figure(figsize=(10, 4))
    plt.subplot(1, 2, 1)
    plt.hist(lengths, bins=10, edgecolor='k', alpha=0.7, color='#5dade2')
    plt.xlabel("Program Length")
    plt.ylabel("Frequency")
    plt.title("Complexity Distribution (Length)")
    
    plt.subplot(1, 2, 2)
    plt.hist(depths, bins=10, edgecolor='k', alpha=0.7, color='#e08214')
    plt.xlabel("Program Depth")
    plt.ylabel("Frequency")
    plt.title("Complexity Distribution (Depth)")
    
    plt.tight_layout()
    plt.show()

In [ ]:
# ====================================
# MAIN EXECUTION
# ====================================

def run_comprehensive_analysis():
    """Run complete SR analysis pipeline"""
    
    print("Starting Comprehensive Symbolic Regression Analysis...")
    print("="*60)
    
    # 1. Basic Visualizations
    print("\n1. Creating basic visualizations...")
    plot_scatter_comparison(y_train_plot, y_pred_train_plot, y_test_plot, y_pred_test_plot)
    plot_residual_analysis(y_test_plot, y_pred_test_plot)
    plot_time_series_comparison(y_test_plot, y_pred_test_plot)
    plot_error_histogram(y_test_plot, y_pred_test_plot)
    
    # 2. Metrics Calculation
    print("\n2. Calculating metrics...")
    train_metrics, test_metrics = print_metrics(y_train_plot, y_pred_train_plot, y_test_plot, y_pred_test_plot)
    
    # 3. Feature Sensitivity Analysis
    print("\n3. Running feature sensitivity analysis...")
    top_indices, sensitivities = plot_sensitivity_analysis(sr, Xtr, feat_names)
    
    # 4. Partial Dependence Plots
    print("\n4. Creating partial dependence plots...")
    plot_pdp_analysis(sr, Xtr, feat_names, top_indices)
    
    # 5. Bootstrap Stability Analysis
    print("\n5. Running bootstrap stability analysis...")
    expr_list, complexities, Yhat_te = bootstrap_stability_analysis(
        sr, Xtr, y_train, Xte, y_test, n_bootstrap=30
    )
    plot_bootstrap_results(y_test, Yhat_te, expr_list, complexities)
    
    print("\n" + "="*60)
    print("Comprehensive analysis completed!")
    
    return {
        'train_metrics': train_metrics,
        'test_metrics': test_metrics,
        'top_features': top_indices,
        'sensitivities': sensitivities,
        'bootstrap_results': (expr_list, complexities, Yhat_te)
    }

# Ana çalıştırma
if __name__ == "__main__":
    results = run_comprehensive_analysis()

In [ ]:
# ==== INFERENCE TIME & FIGURE OF MERIT (ONLY THIS PART) ====
import time
import gc
import sys
import numpy as np

def _fmt_seconds(s):
    # İnsan gibi format: ns/µs/ms/s aralığında otomatik
    if s < 1e-6:
        return f"{s*1e9:.1f} ns"
    if s < 1e-3:
        return f"{s*1e6:.1f} µs"
    if s < 1:
        return f"{s*1e3:.2f} ms"
    return f"{s:.3f} s"

def measure_inference_speed(estimator, X,
                            warmup=50,
                            repeats_single=1000,
                            batch_sizes=(1, 8, 32, 128, None),
                            random_state=42):
    """
    batch_sizes:
      - 1: tek örnek gecikmesi
      - sayılar: mini-batch gecikmesi (N örnek bir seferde)
      - None: X'in tamamı (full-batch)
    """
    rng = np.random.default_rng(random_state)
    X_np = X.values if hasattr(X, "values") else np.asarray(X)
    n = X_np.shape[0]

    # Model karmaşıklığı (gplearn)
    prog = getattr(estimator, "_program", None)
    prog_len = getattr(prog, "length_", None)
    prog_depth = getattr(prog, "depth_", None)

    print("\n" + "="*72)
    print("INFERENCE SPEED & FIGURE OF MERIT")
    print("="*72)
    print(f"Python: {sys.version.split()[0]} | NumPy: {np.__version__}")
    if prog is not None:
        print(f"Expression length (nodes): {prog_len}, depth: {prog_depth}")
        # İstersen formülü de göster:
        # print(f"Expression: {prog}")
    else:
        print("Warning: gplearn program objesi bulunamadı.")

    # --- Warmup (JIT yok ama cache ve memory ayarı için faydalı) ---
    if n > 0:
        idx_warm = rng.integers(0, n, size=min(warmup, max(1, n)))
        for i in idx_warm:
            _ = estimator.predict(X_np[i:i+1])
        _ = estimator.predict(X_np[: min(n, 256)])  # küçük bir batch warmup

    results = []

    for bs in batch_sizes:
        if bs == 1:
            # Tek örnek gecikmesi (repeats_single kez farklı örneklerle)
            times = []
            gc.collect()
            for _ in range(repeats_single):
                i = rng.integers(0, n)
                x1 = X_np[i:i+1]
                t0 = time.perf_counter()
                _ = estimator.predict(x1)
                t1 = time.perf_counter()
                times.append(t1 - t0)
            times = np.array(times)
            mean_t = float(times.mean())
            median_t = float(np.median(times))
            p90 = float(np.percentile(times, 90))
            p99 = float(np.percentile(times, 99))
            thr = 1.0 / mean_t  # samples/sec

            results.append({
                "batch_size": 1,
                "mean_latency_s": mean_t,
                "median_latency_s": median_t,
                "p90_latency_s": p90,
                "p99_latency_s": p99,
                "throughput_sps": thr
            })

            print("\n--- Single-sample latency (batch=1) ---")
            print(f"Mean   : {_fmt_seconds(mean_t)}  | Throughput: {thr:,.0f} samples/s")
            print(f"Median : {_fmt_seconds(median_t)}")
            print(f"P90    : {_fmt_seconds(p90)}")
            print(f"P99    : {_fmt_seconds(p99)}")

        else:
            # Mini-batch veya full-batch
            if bs is None:
                # full-batch: tüm X bir kerede
                bs_eff = n
                if bs_eff == 0:
                    print("\nFull-batch ölçümü atlandı (X boş).")
                    continue
                batch = X_np
                label = "FULL-BATCH"
            else:
                bs_eff = min(bs, n) if n > 0 else 0
                if bs_eff == 0:
                    print(f"\nBatch={bs} ölçümü atlandı (X boş).")
                    continue
                idx = rng.integers(0, n, size=bs_eff)
                batch = X_np[idx]
                label = f"BATCH={bs_eff}"

            # Birkaç tekrar ile istatistik
            repeats = 50 if bs_eff >= 32 else 100
            times = []
            gc.collect()
            for _ in range(repeats):
                t0 = time.perf_counter()
                _ = estimator.predict(batch)
                t1 = time.perf_counter()
                times.append(t1 - t0)
            times = np.array(times)
            mean_t = float(times.mean())
            median_t = float(np.median(times))
            p90 = float(np.percentile(times, 90))
            p99 = float(np.percentile(times, 99))
            thr = bs_eff / mean_t  # samples/sec

            results.append({
                "batch_size": int(bs_eff),
                "mean_latency_s": mean_t,
                "median_latency_s": median_t,
                "p90_latency_s": p90,
                "p99_latency_s": p99,
                "throughput_sps": thr
            })

            print(f"\n--- {label} latency ---")
            print(f"Mean   : {_fmt_seconds(mean_t)}  | Eff. batch: {bs_eff}  | Throughput: {thr:,.0f} samples/s")
            print(f"Median : {_fmt_seconds(median_t)}")
            print(f"P90    : {_fmt_seconds(p90)}")
            print(f"P99    : {_fmt_seconds(p99)}")

    # Özet FoM (figure of merit)
    # - Single-sample median latency
    # - P99 latency
    # - Best throughput (max over all batches)
    one = next((r for r in results if r["batch_size"] == 1), None)
    best_thr = max(results, key=lambda r: r["throughput_sps"]) if results else None

    print("\n" + "-"*72)
    print("FIGURE OF MERIT (FoM)")
    if one:
        print(f"Single-sample median latency : {_fmt_seconds(one['median_latency_s'])}")
        print(f"Single-sample P99 latency    : {_fmt_seconds(one['p99_latency_s'])}")
    if best_thr:
        print(f"Max throughput               : {best_thr['throughput_sps']:,.0f} samples/s "
              f"(batch={best_thr['batch_size']})")
    if prog is not None:
        print(f"Expression complexity        : length={prog_len}, depth={prog_depth}")
    print("-"*72 + "\n")

    return results

# ÇALIŞTIR
_ = measure_inference_speed(sr, X_test)
# İstersen eğitim seti için de:
# _ = measure_inference_speed(sr, X_train)
